In [76]:
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble, Aer, IBMQ, QuantumRegister, AncillaRegister
from qiskit.quantum_info.operators import Operator
from qiskit.quantum_info import Statevector
from qiskit.visualization import array_to_latex

In [77]:
#parameters

M = 8 # lattices

w1 = 0.5 ##diffusion
w2 = 0.5 ##diffusion
e1 = 1   ##right
e2 = -1  ##left
cs = 1   ##speed of sound
u = 0.2  ##advection 


x = w1*(1+e1*u/cs**2)
y = w2*(1+e2*u/cs**2)
print(x,y)
lambda1 = np.round(np.log(complex(x, np.sqrt(1-x**2))), 10)
lambda2 = np.round(np.log(complex(y, np.sqrt(1-y**2))), 10)

0.6 0.4


In [78]:
def oneStep(lambda1,lambda2):
    
    #initializing

    q = QuantumRegister(n,'q')
    a = AncillaRegister(1,'a')

    qc = QuantumCircuit(q)
    qc.add_register(a)
     
    qc.barrier()
    
    
    #collisions
    def createC(lambda1, lambda2, isC1 = True):
    
        if not isC1:
            lambda1=np.conj(lambda1)
            lambda2=np.conj(lambda2)

        lambda1=lambda1.imag
        lambda2=lambda2.imag

        qc=QuantumCircuit(2)

        #phase1 -> x -> phase1 -> x -----ctrl=0
        qc.cp(lambda1, 0 ,1 , ctrl_state = 0)
        qc.cx(0, 1, ctrl_state = 0)
        qc.cp(lambda1, 0, 1, ctrl_state = 0)
        qc.cx(0, 1, ctrl_state = 0)

        #phase2 -> x -> phase2 -> x -----ctrl=1 is default
        qc.cp(lambda2, 0, 1)
        qc.cx(0, 1)
        qc.cp(lambda2, 0, 1)
        qc.cx(0, 1)

        return qc
    
    C1 = createC(lambda1, lambda2, True)
    c1gate = C1.to_gate(label = 'c1')
    C2 = createC(lambda1, lambda2, False)
    c2gate = C2.to_gate(label = 'c2')
    
    ######adding steps to circuit
    qc.h(a)
    
    ##c1 and c2 act on every qubit controlled on ancilla
    for i in range(n-1):
        qc.append(c1gate.control(1, ctrl_state = 0),[a,n-1,i])
    for i in range(n-1):
        qc.append(c2gate.control(1, ctrl_state = 1),[a,n-1,i])

    qc.h(a)
    qc.draw()
    
    
    
    #propagation
    def rshift(n):
        circ = QuantumCircuit(n)
        for i in range(n):
            if i == n-1:
                circ.x(i)
            else:
                circ.mcx(list(range(i+1,n)), i)
        return circ

    def lshift(n):
        circ = QuantumCircuit(n)
        for i in reversed(range(n)):
            if i == n-1:
                circ.x(i)
            else:
                circ.mcx(list(range(i+1,n)), i)
        return circ

    R = rshift(n).to_gate(label = "R").control(1, ctrl_state = 0)
    L = lshift(n).to_gate(label = "L").control(1, ctrl_state = 1)
    
    cbits = [a]
    cbits.extend([i for i in range(n)])  #### could also be [i for i in range(n-1,-1,-1)]

    qc.append(R,cbits)
    qc.append(L,cbits)

    
    ########################## macros
    qc.swap(a,n-1)
    qc.h(a)
    #### -> not sure if there is an extra step
    
    return qc

In [79]:
qc = oneStep(lambda1,lambda2)

In [88]:
classicalState = [0.1 for i in range(2*M)]
classicalState[3] = 0.3
classicalState[3+M] = classicalState[3]

for i in range(M):
    classicalState[i]*=x
for i in range(M,2*M):
    classicalState[i]*=y

for i in range(2*M):
    classicalState[i] = np.sqrt(classicalState[i])### normalization purposes

array_to_latex(classicalState, max_size = 32)

<IPython.core.display.Latex object>

In [85]:
sv = Statevector(classicalState)  
ancillaSV = Statevector([1,0])####need to tensor multiple with ancilla bit
tensored = sv.tensor(ancillaSV)
array_to_latex(tensored, max_size = 32)

<IPython.core.display.Latex object>

In [86]:
#####doing t timesteps
t = 1
for i in range(t):
    tensored = tensored.evolve(qc)

In [87]:
array_to_latex(tensored, max_size = 32)

<IPython.core.display.Latex object>